In [1]:
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
class QuotesLogin(scrapy.Spider):

    name = "login"

    start_urls = [
	    'http://quotes.toscrape.com/login',
	    ]
    
    # parse function for login
    def parse(self, response):
        # FormRequest used to login
        return scrapy.FormRequest.from_response(
            response,
            formdata={'username':'john', 'password': 'secret'},

        # Function to be called once logged in
        callback=self.after_login
        )
    
    # Callback used after login
    def after_login(self, response):

        for quote in response.css('div.quote'):
            yield{
                'text': quote.css('span.text::text').get(),
                'author' :quote.css('span small::text').get(),
                'tags' :quote.css('div.tags a.tag::text').getall(),
            }
        
        # Select the NEXT button and store it in next page
        try:
            next_page = response.css('li.next a').attrib["href"]
        except KeyError:
            # In the last page, there won't be any "href" and a KeyError will be raised
            logging.info('No next page, Terminating crawling process.')
        else:
            # If a next page is found, execute the parse method once again
            yield response.follow(next_page, callback=self.after_login)

In [3]:
filename = "4_quotesauthentication.json"
if filename in os.listdir('src/'):
        os.remove('src/' + filename)
process = CrawlerProcess(settings={
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
        'LOG_LEVEL': logging.INFO,
        "FEEDS":{
                'src/' + filename : {"format":"json"},
        }
})

# Start the crawling using the spider you defined above
process.crawl(QuotesLogin)
process.start()

2024-08-27 17:35:18 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-08-27 17:35:18 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.7.0, Python 3.12.4 (main, Jun  6 2024, 18:26:44) [Clang 15.0.0 (clang-1500.3.9.4)], pyOpenSSL 24.2.1 (OpenSSL 3.3.1 4 Jun 2024), cryptography 43.0.0, Platform macOS-14.6.1-arm64-arm-64bit
2024-08-27 17:35:18 [scrapy.addons] INFO: Enabled addons:
[]
2024-08-27 17:35:18 [py.warnings] WARNING: /Users/marie-sophiechenevier/Library/CloudStorage/Dropbox/2024-DataScience/GitHub/2-Jedha_Fullstack/.venv/lib/python3.12/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reas